In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("yarn")\
.appName("ch29Clustering").getOrCreate()

#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "1g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "4")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [3]:
sc = spark.sparkContext

In [4]:
for x in sc._conf.getAll():
    if "/proxy/" in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_18510


In [6]:
data_dir = "/user/kranthidr/dataSets/spark-guide/"
path = data_dir + "retail-data/by-day/*.csv"

In [11]:
sales = va.transform(spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(path)
  .limit(50)
  .coalesce(4)
  .where("Description IS NOT NULL"))

In [12]:
sales.show(10)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|   features|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   573319|    23200|     JUMBO BAG PEARS|      20|2011-10-30 10:18:00|     2.08|   17651.0|United Kingdom|[20.0,2.08]|
|   573319|    23203|JUMBO BAG VINTAGE...|      10|2011-10-30 10:18:00|     2.08|   17651.0|United Kingdom|[10.0,2.08]|
|   573319|    22947|WOODEN ADVENT CAL...|       6|2011-10-30 10:18:00|     7.95|   17651.0|United Kingdom| [6.0,7.95]|
|   573319|    21262|WHITE GOOSE FEATH...|       6|2011-10-30 10:18:00|     2.95|   17651.0|United Kingdom| [6.0,2.95]|
|   573319|    21261|GREEN GOOSE FEATH...|       6|2011-10-30 10:18:00|     2.95|   17651.0|United Kingdom| [6.0,2.95]|
|   573319|   84341B|SMALL PINK MAGIC ..

In [13]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler()\
  .setInputCols(["Quantity", "UnitPrice"])\
  .setOutputCol("features")

In [14]:
sales.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, features: vector]

In [17]:
# COMMAND ----------
from pyspark.ml.clustering import KMeans
km = KMeans().setK(5)

# .. versionadded:: 1.5.0
# Init docstring: __init__(self, 
#                          featuresCol="features", predictionCol="prediction", k=2,                  
#                          initMode="k-means||", initSteps=2, tol=1e-4, maxIter=20, seed=None)


In [18]:
print km.explainParams()

featuresCol: features column name. (default: features)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2, current: 5)
maxIter: max number of iterations (>= 0). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: -7649703878154674547)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)


In [19]:
kmModel = km.fit(sales)

In [20]:
# COMMAND ----------

summary = kmModel.summary

In [23]:
print summary.clusterSizes # number of points

[8, 12, 17, 11, 2]


In [24]:
kmModel.computeCost(sales)

309.9689161987512

In [25]:
centers = kmModel.clusterCenters()

In [26]:
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 2.5     11.24375]
[11.33333333  1.1       ]
[4.88235294 3.95176471]
[24.36363636  0.94636364]
[48.    1.32]


In [32]:
# COMMAND ----------
from pyspark.ml.clustering import BisectingKMeans
bkm = BisectingKMeans().setK(5).setMaxIter(5)
bkmModel = bkm.fit(sales)

# Init docstring: __init__(self, 
#                          featuresCol="features", predictionCol="prediction", maxIter=20,                  
#                          seed=None, k=4, minDivisibleClusterSize=1.0)

In [33]:
# COMMAND ----------

summary = bkmModel.summary

In [34]:
print summary.clusterSizes # number of points

[8, 16, 13, 10, 3]


In [35]:
kmModel.computeCost(sales)

309.9689161987512

In [36]:
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 2.5     11.24375]
[11.33333333  1.1       ]
[4.88235294 3.95176471]
[24.36363636  0.94636364]
[48.    1.32]


In [40]:
# COMMAND ----------

from pyspark.ml.clustering import GaussianMixture
gmm = GaussianMixture().setK(5)

# Init docstring: __init__(self, 
#                          featuresCol="features", predictionCol="prediction", k=2,                  
#                          probabilityCol="probability", tol=0.01, maxIter=100, seed=None)

In [41]:
print gmm.explainParams()

featuresCol: features column name. (default: features)
k: Number of independent Gaussians in the mixture model. Must be > 1. (default: 2, current: 5)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
seed: random seed. (default: -8733409933116740608)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.01)


In [42]:
model = gmm.fit(sales)

In [43]:
# COMMAND ----------

summary = model.summary
print model.weights

[0.1809062831871321, 0.07999196088784609, 0.26043554453499296, 0.1999999297055438, 0.278666281684485]


In [44]:
model.gaussiansDF.show()

+--------------------+--------------------+
|                mean|                 cov|
+--------------------+--------------------+
|[6.64564026112604...|0.933953258142771...|
|[1.99989950106789...|0.499949740950633...|
|[27.9651724132995...|86.7354745185911 ...|
|[12.0000000000027...|1.197804943230182...|
|[3.64113652437146...|0.373621349812460...|
+--------------------+--------------------+



In [45]:
summary.cluster.show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         0|
|         2|
|         0|
|         2|
|         4|
|         4|
|         4|
|         1|
|         0|
|         0|
|         3|
|         0|
|         2|
|         2|
|         4|
|         4|
+----------+
only showing top 20 rows



In [46]:
summary.clusterSizes

[9, 4, 13, 10, 14]

In [47]:
summary.probability.show()

+--------------------+
|         probability|
+--------------------+
|[4.64645093967825...|
|[4.42105054057479...|
|[7.44248887777602...|
|[3.14495872190316...|
|[0.99997796565732...|
|[3.37944930887881...|
|[0.99999999999951...|
|[8.43073641486631...|
|[0.00752831083286...|
|[0.00752831083286...|
|[0.00227494800777...|
|[2.45617379108935...|
|[0.99361333095573...|
|[0.99361333095573...|
|[3.66470080547568...|
|[0.99172939115085...|
|[3.96818549340122...|
|[3.14495872190316...|
|[0.00752831083286...|
|[0.00752831083286...|
+--------------------+
only showing top 20 rows



In [48]:
# COMMAND ----------

from pyspark.ml.feature import Tokenizer, CountVectorizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.drop("features"))

In [50]:
cv = CountVectorizer()\
  .setInputCol("DescOut")\
  .setOutputCol("features")\
  .setVocabSize(500)\
  .setMinTF(0)\
  .setMinDF(0)\
  .setBinary(True)

# Init docstring: __init__(self, 
#                          minTF=1.0, minDF=1.0, vocabSize=1 << 18, binary=False, inputCol=None,                 
#                          outputCol=None)

In [51]:
cvFitted = cv.fit(tokenized)

In [52]:
prepped = cvFitted.transform(tokenized)

In [53]:
# COMMAND ----------

from pyspark.ml.clustering import LDA

In [59]:
lda = LDA().setK(10).setMaxIter(5)


# Init docstring: __init__(self, 
#                          featuresCol="features", maxIter=20, seed=None, checkpointInterval=10,                  
#                          k=10, optimizer="online", learningOffset=1024.0, learningDecay=0.51,                  
#                          subsamplingRate=0.05, optimizeDocConcentration=True,                  
#                          docConcentration=None, topicConcentration=None,                  
#                          topicDistributionCol="topicDistribution", keepLastCheckpoint=True):

In [60]:
print lda.explainParams()

checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
docConcentration: Concentration parameter (commonly named "alpha") for the prior placed on documents' distributions over topics ("theta"). (undefined)
featuresCol: features column name. (default: features)
k: The number of topics (clusters) to infer. Must be > 1. (default: 10, current: 10)
keepLastCheckpoint: (For EM optimizer) If using checkpointing, this indicates whether to keep the last checkpoint. If false, then the checkpoint will be deleted. Deleting the checkpoint can cause failures if a data partition is lost, so set this bit with care. (default: True)
learningDecay: Learning rate, set as anexponential decay rate. This should be between (0.5, 1.0] to guarantee asymptotic convergence. (default: 0.51)
learningOffset: A (pos

In [61]:
model = lda.fit(prepped)

In [62]:
# COMMAND ----------
model.describeTopics(3).show()

+-----+---------------+--------------------+
|topic|    termIndices|         termWeights|
+-----+---------------+--------------------+
|    0|  [63, 113, 17]|[0.01164489032906...|
|    1|   [10, 84, 32]|[0.00896269795055...|
|    2|    [8, 14, 80]|[0.00906706283933...|
|    3|  [16, 112, 82]|[0.00891670853537...|
|    4|     [2, 4, 58]|[0.01771558726789...|
|    5|    [9, 13, 66]|[0.01168320143030...|
|    6|   [81, 94, 34]|[0.00879897934157...|
|    7|[117, 131, 114]|[0.00869931727630...|
|    8|   [71, 80, 47]|[0.01201947260119...|
|    9|  [32, 42, 138]|[0.00919017536197...|
+-----+---------------+--------------------+



In [63]:
cvFitted.vocabulary
# COMMAND ----------

[u'water',
 u'hot',
 u'vintage',
 u'bottle',
 u'paperweight',
 u'6',
 u'home',
 u'doormat',
 u'landmark',
 u'bicycle',
 u'frame',
 u'ribbons',
 u'',
 u'classic',
 u'rose',
 u'kit',
 u'leaf',
 u'sweet',
 u'bag',
 u'airline',
 u'doorstop',
 u'light',
 u'in',
 u'christmas',
 u'heart',
 u'calm',
 u'set',
 u'keep',
 u'balloons',
 u'night',
 u'lights',
 u'12',
 u'tin',
 u'english',
 u'caravan',
 u'stuff',
 u'tidy',
 u'oxford',
 u'full',
 u'cottage',
 u'notting',
 u'drawer',
 u'mushrooms',
 u'chrome',
 u'champion',
 u'amelie',
 u'mini',
 u'the',
 u'giant',
 u'design',
 u'elegant',
 u'tins',
 u'jet',
 u'fairy',
 u"50's",
 u'holder',
 u'message',
 u'blue',
 u'storage',
 u'tier',
 u'covent',
 u'world',
 u'skulls',
 u'font',
 u'hearts',
 u'skull',
 u'clips',
 u'bell',
 u'red',
 u'party',
 u'chalkboard',
 u'save',
 u'4',
 u'coloured',
 u'poppies',
 u'garden',
 u'nine',
 u'girl',
 u'shimmering',
 u'doughnut',
 u'dog',
 u'3',
 u'tattoos',
 u'chilli',
 u'coat',
 u'torch',
 u'sunflower',
 u'tale',
 u'